In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import re
import json
import numpy as np
from secretkeys import MY_WORDSAPI_KEY

In [2]:
with open('../data/mit_ideonomy_traits.json', 'r') as f:
    mit_ideonomy_traits = json.load(f)

In [3]:
with open("../data/update.txt") as f:
    yihao_traits = f.readlines()

In [4]:
mit_set = set()
for key in mit_ideonomy_traits.keys():
    for trait in mit_ideonomy_traits[key]:
        mit_set.add(trait.lower())

In [5]:
yihao_set = set()
for trait in yihao_traits:
    yihao_set.add(trait.strip().lower())

In [6]:
common_set = mit_set & yihao_set
print(len(mit_set))
print(len(yihao_set))
print(len(common_set))

637
728
164


In [7]:
url_list = []
for trait in yihao_set:
    url = f"https://wordsapiv1.p.rapidapi.com/words/{trait}/definitions"
    url_list.append(url)

In [8]:
url_list

['https://wordsapiv1.p.rapidapi.com/words/large/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/tidy/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/doltish/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/intimate/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/disorderly/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/inactive/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/affirmatory/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/infirm/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/discrepant/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/undaunted/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/vocal/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/courteous/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/gracious/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/dashing/definitions',
 'https://wordsapiv1.p.rapidapi.com/words/egotistic/definitions',
 'https://wordsapiv1.p.rapida

In [9]:
class DefinitionSpider(scrapy.Spider):   
    name = 'definition_spider'
    start_urls = url_list  # assign urls we're interested in here! should put in cls constructor next time.
    
    def start_requests(self):
        cls = self.__class__
        if not self.start_urls and hasattr(self, 'start_url'):
            raise AttributeError(
                "Crawling could not start: 'start_urls' not found "
                "or empty (but found 'start_url' attribute instead, "
                "did you miss an 's'?)")
        else:
            for url in self.start_urls:
                yield scrapy.http.Request(url, headers = {'x-rapidapi-host': "wordsapiv1.p.rapidapi.com",\
                                                          'x-rapidapi-key': MY_WORDSAPI_KEY})
    def parse(self, response):
        jsonresponse = json.loads(response.text)
        yield jsonresponse
        
#         global blob2
#         blob1 = response.xpath(".//div[starts-with(@class, 'sb has-num')]")
#         blob2 = blob1.xpath("//span[@class='dtText']/text()").getall()  # gets a lot of 
#         # redundant data here. not sure how to prevent that...

#         defs = [x.strip() for x in blob2]
#         unique_defs = np.unique(defs)  # rmv redundant data
#         unique_defs = [x for x in unique_defs if x != "" and x != ","]  # rmv blank lines
#         print(f'num of unique defs: {len(unique_defs)}')
        
#         yield {trait: unique_defs}
        

In [10]:
def main():
    save_path = input("enter path to save file:\n")
    process = CrawlerProcess(settings={
        "FEEDS": {
            save_path: {"format": "json"},
        },
    })

    process.crawl(DefinitionSpider)
    process.start()  # the script will block here until the crawling is finished
    
if __name__ == '__main__':
    main()

enter path to save file:
 ../data/wordsapi_yihao_output.json


2020-07-30 00:12:23 [scrapy.utils.log] INFO: Scrapy 2.1.0 started (bot: scrapybot)
2020-07-30 00:12:23 [scrapy.utils.log] INFO: Versions: lxml 4.5.1.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Windows-10-10.0.17763-SP0
2020-07-30 00:12:23 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-07-30 00:12:23 [scrapy.crawler] INFO: Overridden settings:
{}
2020-07-30 00:12:23 [scrapy.extensions.telnet] INFO: Telnet Password: 4c5d3d2ad9a47072
2020-07-30 00:12:23 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2020-07-30 00:12:23 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloader